<img align = 'center' src="../../Images/ML_SWE.jpg" alt = '.../Images/' width = '1000'/>

## Prediction Simulation for Water Year 2019

We will be testing and evaluating the performance of the model over the 2019 WY at select locations

For example, the operational capacity of the model should begin at the beginning of the water year (WY week 1) with a previous SWE value of 0-in for each 1-km grid.
Model evaluation could then assess the forecasting skill on a hold-out dataset of an entire year in which known neighboring values do not control predictions, but rather driving to-date predictions from the predicted previous SWE values. 
Due to the high model performance, future work will target the assessment of model skill with an operational motivation

## Model Training/Testing influence on Model Results.

The model training/testing partitioning methodology has a strong influence on model performance and the goal of model evaluation.
The objective of the modeling effort was to examine the spatial extrapolation capacity of the model from selected monitoring stations to the overall region, best suited to a 75/25% training/testing split, respectively.
While it is critical to address the strong serial correlation in SWE accumulation and melt throughout the season, the high correlation between weeks has the potential to inflate model skill when using a 75/25% training/testing split due to the previous SWE feature being known.
An assessment of the operational capacity of the model is different than assessing the ability to extrapolate regional SWE from in-situ monitoring stations.
For example, the operational capacity of the model should begin at the beginning of the water year (WY week 1) with a previous SWE value of 0-in for each 1-km grid.
Model evaluation could then assess the forecasting skill on a hold-out dataset of an entire year in which known neighboring values do not control predictions, but rather driving to-date predictions from the predicted previous SWE values. 
Due to the high model performance, future work will target the assessment of model skill with an operational motivation.

In [1]:
import os
import sys
sys.path.insert(0, '..') #sys allows for the .ipynb file to connect to the shared folder files
from shared_scripts import Hindcast_Initialization, NSM_SCA

#Set working directories
cwd = os.getcwd()
datapath = f"{os.path.expanduser('~')}/SWEML" 

2024-03-04 21:06:27.574464: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-04 21:06:27.579136: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-04 21:06:27.579157: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Bring prediction DFs in
The Prediction DFs do not store after a git push and this will save your results, make sure you have the correct model name in the function

In [2]:
# frequency = 'Weekly'
# Hindcast_Initialization.AWS_to_Hindcast('Neural_Network', frequency)

# Initiate Simulation

You can run all regions or just ones for development, not that only the regions entered below will work for the Hindcast Evaluator

In [3]:
new_year = '2019'
threshold = '20.0' #This threshold is standardized for now, to recalculate see Dr. Johnson
Region_list = [ 'N_Sierras', 'S_Sierras_High', 'S_Sierras_Low','Greater_Yellowstone', 
    'N_Co_Rockies', 'SW_Mont', 'SW_Co_Rockies', 'GBasin', 'N_Wasatch', 'N_Cascade',
     'S_Wasatch', 'SW_Mtns', 'E_WA_N_Id_W_Mont', 'S_Wyoming', 'SE_Co_Rockies', 
     'Sawtooth', 'Ca_Coast', 'E_Or', 'N_Yellowstone', 'S_Cascade', 'Wa_Coast',
     'Greater_Glacier', 'Or_Coast'
              ]

frequency = 'Daily' #enter Weekly or Daily



datelist = Hindcast_Initialization.Hindcast_Initialization(cwd, datapath, new_year, threshold, Region_list, frequency)

Creating files for a historical simulation within  'N_Sierras', 'S_Sierras_High', 'S_Sierras_Low', 'Greater_Yellowstone', 'N_Co_Rockies', 'SW_Mont', 'SW_Co_Rockies', 'GBasin', 'N_Wasatch', 'N_Cascade', 'S_Wasatch', 'SW_Mtns', 'E_WA_N_Id_W_Mont', 'S_Wyoming', 'SE_Co_Rockies', 'Sawtooth', 'Ca_Coast', 'E_Or', 'N_Yellowstone', 'S_Cascade', 'Wa_Coast', 'Greater_Glacier', 'Or_Coast'  regions for water year  2019


## Run your SSM in hindcast mode to evaluate operational capacity.



In [ ]:
#run the model through all time (data acqusition already completed)
model = 'Neural_Network'
frequency = 'Daily'

for day in datelist:
    print('Updating SWE predictions for ', day)
    #connect interactive script to Wasatch Snow module
    Snow = NSM_SCA.NSM_SCA(day, threshold=threshold, Regions = Region_list, modelname = model, frequency = frequency)
    
    #Go get SNOTEL observations - all data currently loaded, set to True to download
    #Snow.Get_Monitoring_Data_Threaded(getdata = True)

    #Initialize/Download the granules, all data preprocessed for the SSM activRegion_listhange to True to use the functions.
    Snow.initializeGranules(getdata = True)

    #Process observations into Model prediction ready format,
    Snow.Data_Processing(SCA = True)

    #Agument with SCA
    Snow.augmentPredictionDFs()

    #Make predictions, set NewSim to False
    Snow.SWE_Predict(NewSim = True) #Look to multiprocess, each region can do a prediction to speed things up. set NewSim to true for New simulation, turn to false once all data has been proces and saved.

Updating SWE predictions for  2018-10-01
Getting VIIRS fSCA data and calculating the spatial average NSDI
VIIRS fSCA files found locally


100%|██████████| 3/3 [00:00<00:00, 1852.88it/s]


Regional data QA/QC


100%|██████████| 22/22 [00:17<00:00,  1.28it/s]


Calculating mean SCA for each geometry in each region...


...

100%|██████████| 3/3 [00:00<00:00, 1895.87it/s]


Regional data QA/QC


100%|██████████| 22/22 [00:16<00:00,  1.35it/s]


Calculating mean SCA for each geometry in each region...


  9%|▊         | 2/23 [00:11<01:50,  5.28s/it]

### Model Hindcast complete

Lets see how your model performs within SSWEET in the [Hindcast_Evaluation](./Hindcast_Evaluation.ipynb) notebook.

In [5]:
modelname= 'Neural_Network'
folderpath = 'Predictions/Hold_Out_Year/Daily/'
AWSpath = f"Hold_Out_Year/Daily/"
type = '.pkl'
Hindcast_Initialization.Hindcast_to_AWS(modelname, folderpath, AWSpath, type)

['Prediction_DF_2019-03-20.pkl', 'Prediction_DF_2019-03-28.pkl', 'Prediction_DF_SCA_2019-02-10.pkl', 'Prediction_DF_SCA_2018-10-09.pkl', 'Prediction_DF_SCA_2018-10-05.pkl', 'Prediction_DF_SCA_2019-03-19.pkl', 'Prediction_DF_2019-02-14.pkl', 'Prediction_DF_SCA_2019-05-09.pkl', 'Prediction_DF_2019-01-30.pkl', 'Prediction_DF_2019-03-04.pkl', 'Prediction_DF_SCA_2019-02-17.pkl', 'Prediction_DF_2019-04-16.pkl', 'Prediction_DF_SCA_2018-10-23.pkl', 'Prediction_DF_2019-06-15.pkl', 'Prediction_DF_2019-03-17.pkl', 'Prediction_DF_SCA_2018-11-03.pkl', 'Prediction_DF_2018-12-08.pkl', 'Prediction_DF_SCA_2019-06-20.pkl', 'Prediction_DF_SCA_2019-03-01.pkl', 'Prediction_DF_2019-06-03.pkl', 'Prediction_DF_2018-10-13.pkl', 'Prediction_DF_2019-05-13.pkl', 'Prediction_DF_SCA_2018-11-27.pkl', 'Prediction_DF_2019-04-27.pkl', 'Prediction_DF_2018-12-23.pkl', 'Prediction_DF_2019-02-03.pkl', 'Prediction_DF_SCA_2019-03-06.pkl', 'Prediction_DF_SCA_2018-10-01.pkl', 'Prediction_DF_SCA_2018-11-22.pkl', 'Prediction_DF_

100% |########################################################################|
